# Spinning Top (Kreisel) - Chapter 21.4 Exercise

Simulate a spinning top using distance constraints. Three masses connected by rigid distance constraints form a spinning body. Initial angular momentum keeps it spinning despite gravity.

In [2]:
import sys, os
os.add_dll_directory(r"C:\msys64\ucrt64\bin")
sys.path.insert(0, r"C:\Users\fabia\Personal\Uni\Shared\scicomp\schaf3\build\mechsystem")

from mass_spring import *
import numpy as np
from pythreejs import *

In [ ]:
# Create spinning top (Kreisel) with 3 masses connected by distance constraints
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# Create a fixed center point (the pivot)
center_fix = mss.add(Fix((0, 0, 2.0)))

# Create 3 masses arranged in an equilateral triangle in the xy-plane
# centered at height z=2, with center of mass at origin
mass_val = 1.0
radius = 0.5  # radius of triangle from center (reduced for stability)

# Three positions forming equilateral triangle
pos1 = (radius * np.cos(0*2*np.pi/3), radius * np.sin(0*2*np.pi/3), 2.0)
pos2 = (radius * np.cos(1*2*np.pi/3), radius * np.sin(1*2*np.pi/3), 2.0)
pos3 = (radius * np.cos(2*2*np.pi/3), radius * np.sin(2*2*np.pi/3), 2.0)

m1 = mss.add(Mass(mass_val, pos1))
m2 = mss.add(Mass(mass_val, pos2))
m3 = mss.add(Mass(mass_val, pos3))

# Connect the three masses with distance constraints (rigid triangle)
side_length = 2.0 * radius * np.sin(np.pi/3)  # side length of equilateral triangle
mss.add(DistanceConstraint(side_length, (m1, m2)))
mss.add(DistanceConstraint(side_length, (m2, m3)))
mss.add(DistanceConstraint(side_length, (m3, m1)))

# Add distance constraints from each mass to the center (pivot point)
# This keeps the spinning body from flying away
mss.add(DistanceConstraint(radius, (m1, center_fix)))
mss.add(DistanceConstraint(radius, (m2, center_fix)))
mss.add(DistanceConstraint(radius, (m3, center_fix)))

print("Spinning top created with:")
print(f"  Fixed center (pivot) at (0, 0, 2.0)")
print(f"  3 masses of {mass_val} kg each")
print(f"  Arranged in equilateral triangle with side length {side_length:.3f} m")
print(f"  3 distance constraints forming rigid triangle")
print(f"  3 distance constraints anchoring each mass to center (distance {radius} m)")
print(f"  Total constraints: {len(mss.constraints)}")


Spinning top created with:
  3 masses of 1.0 kg each
  Arranged in equilateral triangle with side length 0.866 m
  Center of mass at z=2.0 m
  3 distance constraints forming rigid triangle


In [4]:
# Set initial velocities for spinning motion
# Create angular velocity around z-axis (vertical axis)
omega = 10.0  # angular velocity (rad/s)

print(f"Setting initial angular velocity: {omega:.1f} rad/s around z-axis\n")

for i, m in enumerate(mss.masses):
    x, y, z = m.pos
    # Velocity tangent to circle in xy-plane
    # v = omega × r: for rotation around z-axis, v = omega * (-y, x, 0)
    vx = -omega * y
    vy = omega * x
    vz = 0.0
    # Set velocity directly on the mass
    m.vel = (vx, vy, vz)
    print(f"Mass {i+1} at ({x:6.3f}, {y:6.3f}, {z:6.3f}): velocity ({vx:7.3f}, {vy:7.3f}, {vz:7.3f})")

print(f"\nSystem ready for simulation!")
print(f"Total constraints: {len(mss.constraints)}")


Setting initial angular velocity: 10.0 rad/s around z-axis

Mass 1 at ( 0.500,  0.000,  2.000): velocity ( -0.000,   5.000,   0.000)
Mass 2 at (-0.250,  0.433,  2.000): velocity ( -4.330,  -2.500,   0.000)
Mass 3 at (-0.250, -0.433,  2.000): velocity (  4.330,  -2.500,   0.000)

System ready for simulation!
Total constraints: 3


In [5]:
# Create visualization
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.15, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.1, 16, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

# Create edges connecting the masses (constraints)
constraint_positions = []
for c in mss.constraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraint_positions.append([pA, pB])

constraint_geo = LineSegmentsGeometry(positions=constraint_positions)
constraint_mat = LineMaterial(linewidth=2, color='yellow')
constraints = LineSegments2(constraint_geo, constraint_mat)

# Create axes for reference
axes = AxesHelper(2)

# Scene setup
view_width = 800
view_height = 600

camera = PerspectiveCamera(position=[3, 3, 3], aspect=view_width/view_height)
key_light = DirectionalLight(position=[5, 5, 5])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, constraints, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, …

In [6]:
# Simulate the spinning top
# Watch as gravity causes precession due to angular momentum
from time import sleep

dt = 0.001  # very small time step for stability with constraints
substeps = 100  # many solver iterations to help convergence

print("Starting simulation... watch the top precess!")
print(f"Time step: {dt} s, Solver steps: {substeps}")
print("This may take a while...\n")

failed_steps = 0
for step in range(20000):
    try:
        # Simulate
        mss.simulate(dt, substeps)
    except ValueError as e:
        failed_steps += 1
        print(f"Step {step}: {e}")
        if failed_steps > 10:
            print("Too many convergence failures, stopping.")
            break
        continue
    
    # Update visualization
    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])
    
    # Update constraint edges
    constraint_positions = []
    for c in mss.constraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        constraint_positions.append([pA, pB])
    
    constraints.geometry = LineSegmentsGeometry(positions=constraint_positions)
    
    # Print progress
    if step % 2000 == 0 and step > 0:
        print(f"Step {step}, t = {step*dt:.2f} s, Convergence failures: {failed_steps}")
    
    sleep(0.001)

print(f"Simulation complete! Total convergence failures: {failed_steps}")


Starting simulation... watch the top precess!
Time step: 0.001 s, Solver steps: 100
This may take a while...

Step 2000, t = 2.00 s, Convergence failures: 0


KeyboardInterrupt: 